In [1]:
import matplotlib
# matplotlib.use('Agg')

%load_ext autoreload
%autoreload 2

%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import os

import glob2
from sklearn.decomposition import PCA
import pycorrelate


# visualize results module
from Visualize import Visualize, get_sessions, load_trial_times_whole_stack, get_lever_offset_seconds

from utility_DLC import *


# manually add labels from DLC
from tqdm import tqdm, trange
import scipy

# 
import glob

import umap

# 
#data_dir = '/media/cat/4TBSSD/yuki/'
data_dir = '/media/cat/4TBSSD/yuki/'


labels = ['left_paw', 
'right_paw',
'nose',
'jaw',
'right_ear',
'tongue',
'lever']



Autosaving every 180 seconds


In [17]:
########################################       
########################################       
########################################
def get_satiety_scatter_plots(animal_id,
                              plotting=False):
    A = Analyze()
    A.root_dir = '/media/cat/1TB/yuki/'
    A.lockout = 1  
    A.animal_names = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']
    A.animal_ids = np.arange(len(A.animal_names))

    # 
    import scipy.stats

    width1 = 1
    width2 = 10
    linewidth = 3
    offset1 = 200
    offset2 = 10
    offset3 = 250
    fontsize = 16
    dotsize = 2
    alpha = .15

    bins = np.arange(0,1300,width1)

    pvals = [0.05, 0.01, 0.001, 0.0001, 0.00001]
    cmap = matplotlib.cm.get_cmap('magma')

    #             
    animal_name = A.animal_names[animal_id]
    #for animal_id in ['AQ2']:
    print ("animal_nam: ", animal_name)
    fname = A.root_dir + animal_name + '/'+animal_name+ '_satiety_consumption_all_sessions.npz'
    #print (fname)
    d = np.load(fname,
                allow_pickle=True)
        
    s02 = d['s02']
    s04 = d['s04']


    #############################################
    #############################################
    #############################################
    if plotting:
        fig=plt.figure(figsize=(10,10))
    img1 = []
    img2 = []
    all_sums1 = []
    all_sums2 = []
    vsums1 = []
    vsums2 = []
    
    # 
    all1 = []
    all2 = []
    for k in range(len(s04)):
        temp = s04[k]
        hist = np.histogram(temp, bins = bins)
        temp2 = s02[k]
        hist2 = np.histogram(temp2, bins = bins)
        
        all1.append(temp)
        all2.append(temp2)
        
        # 
        s1 = hist[0].sum()
        if s1==0:
            h1 = np.zeros(hist[0].shape[0])#+0.01
        else:
            h1 = hist[0]/np.max(hist[0])

        # 
        s2 = hist2[0].sum()
        if s2==0:
            h2 = np.zeros(hist2[0].shape[0])#+0.01
        else:
            h2 = hist2[0]/np.max(hist2[0])

        # 
        if s1 ==0 and s2==0:
            continue

        # 
        all_sums1.append(len(temp))
        all_sums2.append(len(temp2))
        

        if True:
            sigma = 3
            h1 = scipy.ndimage.gaussian_filter1d(h1, sigma)
            h2 = scipy.ndimage.gaussian_filter1d(h2, sigma)

        # 
        vsums1.extend(temp)
        vsums2.extend(temp2)

        # 
        img1.append(h1)
        img2.append(h2)

    all_sums1 = np.array(all_sums1)
    img1 = np.vstack(img1)
    rewarded_scatter = all_sums1.copy()

    ###################################################
    ############ CODE 04 RASTER/IMG PLOT ##############
    ###################################################
    if plotting:
        ax=plt.subplot(121)
        plt.imshow(img1, 
                   aspect='auto', 
                   cmap='Blues',
                   interpolation='none')

        plt.xticks([])
    
    ###################################################
    ############ CODE 04 VERTICAL HISTOGRAM ###########
    ###################################################    
    rewarded_vertical_histogram1 = all_sums1.copy()
    if plotting:

        plt.plot(1300-all_sums1/np.max(all_sums1)*offset1+ offset3,
             np.arange(len(all_sums1)),
             linewidth = linewidth,
             c='blue',
             alpha=.5)
    
        plt.xlim(0,1300+offset3)

    ###################################################
    ############ CODE 04 SCATTER PLOT #################
    ###################################################
    vsums1 = np.array(vsums1)
    vsums1 = vsums1.flatten()
    #print ("vsums1: ", vsums1.shape)

    # 
    bins2 = np.arange(0,1300,width2)
    y = np.histogram(vsums1, bins=bins2)
    vsums1 = np.float32(y[0])

    # 
    #vsums1 = vsums1/np.nanmax(vsums1)*offset2+len(all_sums1)
    x = np.arange(y[1][0],y[1][-1],width2)
    y = (vsums1 - np.min(vsums1))/(np.max(vsums1)-np.min(vsums1))*10+len(all_sums1)

    # 
    if plotting:
        plt.scatter(x,
                    y,
                    linewidth = linewidth,
                    s=dotsize,
                    c='blue',
                    alpha=alpha)

    #############################################
    ############ CODE 04 SCATTER FIT ############
    #############################################
    coef = np.polyfit(x,y,1)
    poly1d_fn = np.poly1d(coef) 
    if plotting:
        plt.plot(x, poly1d_fn(x), '--', c='blue',
                linewidth=linewidth)
    rewarded_scatter = [x,y]
    rewarded_fit = [x, poly1d_fn(x).copy()]

    # 
    slope, intercept, r_value, p_value1, std_err = scipy.stats.linregress(x,y)
    #print (r_value, p_value)

    title = ''
    for k in range(len(pvals)):
        if p_value1 < pvals[k]:
            title = title+ "*"

    #plt.title("Rewarded Pulls "+title,fontsize=fontsize)
    if plotting:
        plt.title(title,fontsize=fontsize)
        plt.ylim(-0.5, len(all_sums1)+offset2+1)

    ###################################################
    ################# CODE 02 RASTER IMAGE ############
    ###################################################
    if plotting:
        ax=plt.subplot(122)
    img2 = np.vstack(img2)

    #print ("img2: ", img2.shape)
    if plotting:
        plt.imshow(img2, aspect='auto', 
                   cmap='Reds',
                   interpolation='none')
        plt.xticks([])
        plt.yticks([])

    ###################################################
    ################# CODE 02 VERTICAL HIST ###########
    ###################################################
    
    all_sums2 = np.array(all_sums2)
    rewarded_vertical_histogram2 = all_sums2.copy()
    if plotting:
        plt.plot(1300-all_sums2/np.max(all_sums2)*offset1+ offset3,
                 np.arange(len(all_sums2)),
                 linewidth = linewidth,
                 c='red')
        plt.xlim(0,1300+offset3)

    ###################################################
    ############ CODE 02 SCATTER ######################
    ###################################################
    vsums2 = np.array(vsums2).flatten()

    y = np.histogram(vsums2, bins=bins2)
    vsums2 = np.float32(y[0])

    # 
    x = np.arange(y[1][0],y[1][-1],width2)
    #y = vsums2+len(all_sums2)+1+offset2
    y = (vsums2 - np.min(vsums2))/(np.max(vsums2)-np.min(vsums2))*10+len(all_sums2)

    if plotting:
        plt.scatter(x,
                 y,
                 linewidth = linewidth,
                 s=dotsize,
                 c='red',
                 alpha=alpha)


    # FIT 
    slope, intercept, r_value, p_value2, std_err = scipy.stats.linregress(x,y)
    #print (r_value, p_value)
    title = ''
    for k in range(len(pvals)):
        if p_value2 < pvals[k]:
            title = title+ "*"

    #plt.title("UnRewarded Pulls "+title,fontsize=fontsize)
    if plotting:
        plt.title(title,fontsize=fontsize)

    coef = np.polyfit(x,y,1)
    poly1d_fn = np.poly1d(coef) 
    rewarded_scatter2 = [x,y]
    rewarded_fit2 = [x, poly1d_fn(x).copy()]

    
    # 
    if plotting:
        plt.plot(x, poly1d_fn(x), '--', c='red',
                linewidth=linewidth)

        #
        plt.ylim(-0.5, len(all_sums1)+offset2+1)

    ##############################################
    if plotting:
        if True:
            plt.savefig('/home/cat/'+str(animal_name)+'.png', dpi=300)
            plt.close()
        else:
            plt.show()

    all1 = np.array(all1)
    all2 = np.array(all2)

    return (all1, all2, rewarded_scatter, 
            rewarded_vertical_histogram1, rewarded_vertical_histogram2, 
            rewarded_fit, rewarded_scatter, p_value1, 
            rewarded_fit2, rewarded_scatter2, p_value2)

def clean_traces(data,
                smooth_window = 10):
    
    # 
    for k in trange(data.shape[0]):
        temp = data[k]
        probs = temp[:,2]
        
        threshold = 0.5
        idx = np.where(probs<threshold)[0]
        x = temp[:,0]
        y = temp[:,1]
        x[idx] = np.nan
        y[idx] = np.nan
        
        # 

        for i in range(10000):
            idx = np.where(np.isnan(x))[0]
            #print (idx.shape)
            if idx.shape[0]==0:
                break

            if idx[0]==0:
                idx=idx[1:]
                
            x[idx] = x[idx-1]
            y[idx] = y[idx-1]

        x = scipy.ndimage.median_filter(x, size=smooth_window)
        y = scipy.ndimage.median_filter(y, size=smooth_window)
        #print (x.shape, y.shape, data.shape)
        
        data[k,:,0]=x
        data[k,:,1]=y
    return data
            
    

def plot_velocities(traces, movements, window):
    
    # 
    t=np.linspace(-window, window,window*2*15-1)

    
    fig=plt.figure()
    ax1=plt.subplot(111)
    mean_vel=[]
    for k in trange(movements.shape[0]):
        t_start = int(movements[k]-window)*15
        t_end = int(movements[k]+window)*15

        temp = traces[feature_selected,t_start:t_end,:2]

        if True:
            ax1=plt.subplot(221)
            vel = np.sqrt((temp[1:,0]-temp[:-1,0])**2+
                          (temp[1:,1]-temp[:-1,1])**2)

            mean_vel.append(vel)
            plt.plot(t,vel,c='black',linewidth=2,alpha=.1)

    # complete ax1 plots
    mean_vel = np.array(mean_vel).mean(0)
    ax1.plot(t,mean_vel,c='black',linewidth=5,alpha=1)
    ax1.plot([0,0],[0.1,250],'--',linewidth=5,c='black')
    ax1.set_ylim(0.1,250)
    ax1.set_xlim(t[0],t[-1])
    ax1.semilogy()




def get_positions(movements,
                  traces,
                  colors,
                  plotting=False):
    
    padding_x = 20
    padding_y = 20

    if plotting:
        fig=plt.figure()
        ax1=plt.subplot(111)
    
    # 
    all_traces = []
    for k in trange(movements.shape[0]):
        t_start = int(movements[k]-window)*15
        t_end = int(movements[k]+window)*15

        temp = traces[feature_selected,t_start:t_end,:2]
        if temp.shape[0]==0:
            continue
        
        temp = temp - temp[temp.shape[0]//2]
        temp[:,0] = scipy.ndimage.median_filter(temp[:,0], size=10)
        temp[:,1] = scipy.ndimage.median_filter(temp[:,1], size=10)

        
        # zero out to t=0 point
            
        
        if plotting:
            if True:
                for p in range(1, temp.shape[0]-1,1):
                    plt.plot([temp[p-1,0],temp[p,0]],
                             [temp[p-1,1],temp[p,1]], 
                             c=colors[p],
                             linewidth=3,
                             alpha=.1)
            else:
                plt.plot(temp[:,0], temp[:,1], 
                         c=colors[k],alpha=.4)


            plt.scatter(temp[:,0],temp[:,1], 
                         c=colors, 
                        s=100, alpha=.1)
        
        if temp.shape[0]==(150):
            #print (temp.shape, t_end, t_start)
            all_traces.append(temp)
    
    # complete ax2 plots
    if plotting:
        ax1.set_xlim(ax1.get_xlim()[0]-padding_x,
                     ax1.get_xlim()[1]+padding_x)
        ax1.set_ylim(ax1.get_ylim()[0]-padding_y,
                     ax1.get_ylim()[1]+padding_y)


    all_traces = np.array(all_traces)


    return np.array(all_traces)

# 
def plot_average_positions(mean_trace, 
                           color,
                           fig=None,
                           ax1=None):
    
    #fig = plt.figure()
    if ax1 is None:
        fig=plt.figure()
        ax1=plt.subplot(111)
    #
    mean_trace = mean_trace.mean(0)
    mean_trace[:,0] = scipy.ndimage.median_filter(mean_trace[:,0], size=5)
    mean_trace[:,1] = scipy.ndimage.median_filter(mean_trace[:,1], size=5)

    # 
    if False:
        for p in range(1, mean_trace.shape[0]-1,1):
            ax1.plot([mean_trace[p-1,0],mean_trace[p,0]],
                     [mean_trace[p-1,1],mean_trace[p,1]], 
                     c=color,
                     linewidth=5, alpha=.7)
        
    else:
        ax1.plot(mean_trace[:,0],mean_trace[:,1],
                     c=color,
                     linewidth=5, alpha=.7)
    ax1.set_xticks([])
    ax1.set_yticks([])

    #ax1.scatter(mean_trace[:,0],mean_trace[:,1], 
    #            #c=colors,alpha=1)
    #            c=color,alpha=.1)

#     ax1.set_xlim(np.min(mean_trace[:,0])-1,
#                  np.max(mean_trace[:,0])+1)
#     ax1.set_ylim(np.min(mean_trace[:,1])-1,
#                  np.max(mean_trace[:,1])+1)


def get_movements_lever_pos(trace,
                            times,
                            lockout_window=0):
    
    
    
    
    movement_threshold = 3  # number of pixels per frame inidicating movement from stationarity

    movements = np.zeros((trace.shape[0])) # This tracks any change in movement.

    # compute velocity and median value for 1D data
    vel = trace[1:]-trace[:-1]
           
    idx = np.where(vel<=1)[0]
    vel[idx]=np.nan

    idx2 = np.where(vel>=movement_threshold)[0]  # VELOCITY > min thresh means movement
    movements[idx2]+=1

    # 
    idx = np.where(movements>0)[0]
    print ('idx: ', idx.shape)
    #
    chunks = []
    for p in range(1,idx.shape[0],1):
        if (times[idx[p]]-times[idx[p-1]])>= lockout_window:
            chunks.append([times[idx[p-1]], times[idx[p]]])


    print (labels[k], "  # of quiescent periods: ", len(chunks))
    return np.array(chunks)



def polyfit(x, y, degree=1):
    results = {}

    coeffs = np.polyfit(x, y, degree)

     # Polynomial Coefficients
    results['polynomial'] = coeffs.tolist()

    # r-squared
    p = np.poly1d(coeffs)
    
    # fit values, and mean
    yhat = p(x)                         # or [p(z) for z in x]
    ybar = np.sum(y)/len(y)          # or sum(y)/len(y)
    ssreg = np.sum((yhat-ybar)**2)   # or sum([ (yihat - ybar)**2 for yihat in yhat])
    sstot = np.sum((y - ybar)**2)    # or sum([ (yi - ybar)**2 for yi in y])
    results['determination'] = ssreg / sstot

    # find trends from first to last
    diff = yhat[-1]-yhat[0]
    
    from scipy.stats import pearsonr
 
    # calculate Pearson's correlation
    corr_pearson, _ = pearsonr(y, yhat)
    
    from sklearn.metrics import mean_squared_error
    
    rmse = mean_squared_error(y, yhat)



    return coeffs, ssreg/sstot, corr_pearson, diff, rmse

def get_behavior_scatter_plots(times, 
                               color_id,
                               plotting=True):

    cmaps = ['Blues','Greens',"Oranges", "Reds", "Greys", "Purples"]
    colors = ['blue','green','orange','red','grey','purple']
    
    d = np.load('/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb1_30Hz/IA1pm_Feb1_30Hz_0secNoMove_movements.npz',
               allow_pickle=True)
    
    labels = d['labels']
    print ("labels: ", labels)
    
    # 
    import scipy.stats

    width1 = 1
    width2 = 10
    linewidth = 3
    offset1 = 200
    offset2 = 10
    offset3 = 250
    fontsize = 16
    dotsize = 2
    alpha = .15

    bins = np.arange(0,1300,width1)

    pvals = [0.05, 0.01, 0.001, 0.0001, 0.00001]
    cmap = matplotlib.cm.get_cmap('magma')

    #             
    animal_name = 'IA1'
    print ("animal_nam: ", animal_name)

    s04 = times


    #############################################
    #############################################
    #############################################
    if plotting:
        fig=plt.figure(figsize=(10,10))
    img1 = []
    img2 = []
    all_sums1 = []
    all_sums2 = []
    vsums1 = []
    vsums2 = []
    
    # 
    all1 = []
    all2 = []
    for k in range(len(s04)):
        temp = s04[k]
        hist = np.histogram(temp, bins = bins)
        
        all1.append(temp)
        
        # 
        s1 = hist[0].sum()
        if s1==0:
            h1 = np.zeros(hist[0].shape[0])#+0.01
        else:
            h1 = hist[0]/np.max(hist[0])

        # 
        if s1 ==0:
            continue

        # 
        all_sums1.append(len(temp))
        

        if True:
            sigma = 3
            h1 = scipy.ndimage.gaussian_filter1d(h1, sigma)

        # 
        vsums1.extend(temp)

        # 
        img1.append(h1)

    all_sums1 = np.array(all_sums1)
    try:
        img1 = np.vstack(img1)
    except:
        return (all1, all2, None, 
            None, None, 
            None, None, None, 
            None, None, None)
        
    rewarded_scatter = all_sums1.copy()

    ###################################################
    ############ CODE 04 RASTER/IMG PLOT ##############
    ###################################################
    if plotting:
        ax=plt.subplot(121)
        plt.yticks([])
        plt.imshow(img1, 
                   aspect='auto', 
                   cmap=cmaps[color_id],
                   interpolation='none')

        plt.xticks([])
    
    ###################################################
    ############ CODE 04 VERTICAL HISTOGRAM ###########
    ###################################################    
    rewarded_vertical_histogram1 = all_sums1.copy()
    if plotting:

        plt.plot(1300-all_sums1/np.max(all_sums1)*offset1+ offset3,
             np.arange(len(all_sums1)),
             linewidth = linewidth,
             c=colors[color_id],
             alpha=.5)
    
        plt.xlim(0,1300+offset3)

    ###################################################
    ############ CODE 04 SCATTER PLOT #################
    ###################################################
    vsums1 = np.array(vsums1)
    vsums1 = vsums1.flatten()
    #print ("vsums1: ", vsums1.shape)

    # 
    bins2 = np.arange(0,1300,width2)
    y = np.histogram(vsums1, bins=bins2)
    vsums1 = np.float32(y[0])

    # 
    #vsums1 = vsums1/np.nanmax(vsums1)*offset2+len(all_sums1)
    x = np.arange(y[1][0],y[1][-1],width2)
    y = (vsums1 - np.min(vsums1))/(np.max(vsums1)-np.min(vsums1))*10+len(all_sums1)

    # 
    if plotting:
        plt.scatter(x,
                    y,
                    linewidth = linewidth,
                    s=dotsize,
                    c=colors[color_id],
                    alpha=alpha)

    #############################################
    ############ CODE 04 SCATTER FIT ############
    #############################################
    coef = np.polyfit(x,y,1)
    poly1d_fn = np.poly1d(coef) 
    if plotting:
        plt.plot(x, poly1d_fn(x), '--', c=colors[color_id],
                linewidth=linewidth)
    rewarded_scatter = [x,y]
    rewarded_fit = [x, poly1d_fn(x).copy()]

    # 
    slope, intercept, r_value, p_value1, std_err = scipy.stats.linregress(x,y)
    #print (r_value, p_value)

    title = ''
    for k in range(len(pvals)):
        if p_value1 < pvals[k]:
            title = title+ "*"

    #plt.title("Rewarded Pulls "+title,fontsize=fontsize)
    if plotting:
        plt.title(title,fontsize=fontsize)
        plt.ylim(-0.5, len(all_sums1)+offset2+1)
        plt.xticks([])

    ##############################################
    if plotting:
        if False:
            plt.savefig('/home/cat/'+str(animal_name)+'.png', dpi=300)
            plt.close()
        else:
            plt.show()

    all1 = np.array(all1)
    all2 = np.array(all2)
    rewarded_vertical_histogram2 = rewarded_fit2 = rewarded_scatter2 = p_value2 = None
    
    return (all1, all2, rewarded_scatter, 
            rewarded_vertical_histogram1, rewarded_vertical_histogram2, 
            rewarded_fit, rewarded_scatter, p_value1, 
            rewarded_fit2, rewarded_scatter2, p_value2)




def get_times_array_behavior(animal_id, no_movement):
    ###############################
    animal_names = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']
    animal_id = animal_names[animal_id]

    session_id = 'all'
    sessions = get_sessions(vis.main_dir,
                            animal_id,
                           session_id)

    ###################################### 
    # 
    times_array = []
    for k in range(7):
        times_array.append([])

    for session in sessions:
        #print (session)
        temp_, code_04_times, feature_quiescent = load_trial_times_whole_stack(
                                                                         vis.main_dir,
                                                                         animal_id,
                                                                         session,
                                                                         no_movement)
        # 
        if feature_quiescent is not None:
            #for k in range(len(feature_quiescent)):
            for k in range(7):

                temp = np.array(feature_quiescent[k])
                times_array[k].append(temp)

    times_array = np.array(times_array)

    
    return times_array




def compute_trends_intra_session_behavior(times_array,
                                          selected_behavior,
                                          animal_id,
                                          plot_data):
    
    animal_names = ['M1','M2','M3','M4','M5','M6']

    #
    linewidth=4
    scatter_size = 2
    scale = 15
    if plot_data:
        fig = plt.figure(figsize=(10,5))
    #
    #animal_ids = np.arange(6)
    #animal_ids = [2]

    # 
    #plt.subplot(2,3,animal_id+1)
      #
    (all1, all2, rewarded_scatter, 
     rewarded_vertical_histogram1, rewarded_vertical_histogram2, 
     rewarded_fit, rewarded_scatter, p_value1, 
     rewarded_fit2, rewarded_scatter2, p_value2) = get_behavior_scatter_plots(times_array[selected_behavior], 
                                                                              color_id=selected_behavior,
                                                                              plotting=False)

    d1 = rewarded_fit
    d2 = rewarded_scatter
    p_value = p_value1
    all_ = all1
    
    print ("length all ", len(all_))

    # LOOP OVER WINDOWS OF DATA: 
    window = 3
    width2 = 10

    ctr=0
    for k in range(0,len(all1)-window, 1):
        ctr+=1

    #if data_type == 'rewarded':
    colors = plt.cm.Blues
    cmap=plt.cm.get_cmap('Blues')
    
        
    cmaps = ['Greens','Blues',"Oranges", "Purples", "Greys", "Reds"]
    colors = ['green','blue','orange','purple','grey','red']
    
    
    # 
    colors=cmap(np.arange(len(all_)-window+10)/(len(all_)-window+10))
    coefs = []
    n_trials = []
    for k in range(0,len(all_)-window, 1):
        #print ("hstack: ", all_[k:k+window])
        temp = all_[k:k+window]
        data=[]
        for p in range(len(temp)):
            data.extend(temp[p])
            
        data = np.array(data)
        #print ("data: ", data.shape)
        n_trials.append(data.shape[0])
        #
        bins2 = np.arange(0,1300,width2)
        y = np.histogram(data, bins=bins2)
        xx = y[1][1:]
        yy = y[0]

       # print (xx.shape, yy.shape)
        # FIT 
        coef = np.polyfit(xx,yy,1)
        coefs.append(coef)
        poly1d_fn = np.poly1d(coef) 

        # 
        yy = poly1d_fn(xx)
        yy = yy-yy[0]
        if plot_data:
            plt.plot(xx, yy,  
                     c=colors[k],
                     linewidth=linewidth)

    # 
    if plot_data:
        plt.plot([0,1300],[0,0],'--',c='black',
                 linewidth=2, 
                 alpha=.5)
        #
        plt.title(animal_names[animal_id])
        #plt.ylim(-3, 4.5)
        plt.xlim(0, 1300)
        plt.xticks([])
        plt.yticks([])

        cax = fig.add_axes([0.95, 0.2, 0.02, 0.6])
        cb = matplotlib.colorbar.ColorbarBase(cax, cmap=cmap)    
        plt.suptitle(data_type)
        if False:
            plt.savefig('/home/cat/vertical.png',dpi=300)
            plt.close()
        else:
            plt.show() 
        
    print ("COEFS: ", coefs)
    coefs = np.array(coefs)
    return (coefs, n_trials)



def plot_horizontal_scatter_behavior(times_array,
                                     selected_behavior,
                                     animal_id):
    
    #plotting = False
    animal_names = ['M1','M2','M3','M4','M5','M6']


    labels = ['left_paw', 'right_paw', 'nose', 'jaw', 'right_ear', 'tongue', 'lever']
        
    cmaps = ['Blues','Greens',"Oranges", "Reds", "Greys", "Purples"]
    colors = ['green','blue','orange','cyan','grey','purple']
    
    #
    linewidth=5
    scatter_size = 10
    scale = 15

    plt.xticks([])
    plt.yticks([])
    plt.xlim(0,1300)
    
    #animal_ids = [0]
    #for animal_id in animal_ids:

    #
    (all1, all2, rewarded_scatter, 
     rewarded_vertical_histogram1, rewarded_vertical_histogram2, 
     rewarded_fit, rewarded_scatter, p_value1, 
     rewarded_fit2, rewarded_scatter2, p_value2) = get_behavior_scatter_plots(times_array[selected_behavior], 
                                                                              color_id=selected_behavior,
                                                                              plotting=False)

    d1 = rewarded_fit
    d2 = rewarded_scatter
    p_value = p_value1

    pvals = [0.05, 0.01, 0.001, 0.0001, 0.00001]

    title = ''
    try:
        for k in range(len(pvals)):
            if p_value < pvals[k]:
                title = title+ "*"
    except:
        return
    plt.title(title, fontsize=14, y=.9)

    ######################################## 
    ########### PLOT FIT ################### 
    ######################################## 
    d1[1] = (d1[1]-np.min(d2[1]))#/(np.max(d1[1])-np.min(d1[1]))
    plt.plot(d1[0],d1[1], #+animal_id*scale, 
             #label=animal_names[animal_id]+ " "+title,
             label=animal_names[animal_id]+ " "+title,
             #c=colors[animal_id],
             c=colors[selected_behavior],
             linewidth=linewidth)


    ######################################## 
    ########### PLOT SCATTER ############### 
    ######################################## 
    d2[1] =  (d2[1]-np.min(d2[1]))#/(np.max(d2[1])-np.min(d2[1]))
    plt.scatter(d2[0],d2[1], #+animal_id*scale,
                linestyle='--',
                alpha=.2,
#                     edgecolor=colors[animal_id],
#                     facecolor = colors[animal_id],
                s=scatter_size,
                c=colors[selected_behavior],
                linewidth=linewidth)

    plt.plot([0,1300] , [np.mean(d2[1]), np.mean(d2[1])],
           '--', c='black',linewidth=2,
           alpha=.5)
    # 

    #plt.ylim(0,(animal_id+2)*scale)
   # if plot_legend:
   #     plt.legend(fontsize=3,ncol=2, loc='upper right')  

        


In [18]:
##########################
####### INITIALIZE #######
##########################
# LEVER PULL
vis = Visualize()

# lever-related data
vis.main_dir = data_dir
vis.random_flag = False  # shuffle data to show baseline

# 
vis.window = 15
vis.lockout_window = 10
vis.lockout = False

vis.pca_var = 0.95
vis.pca_flag = True


vis.significance = 0.05
vis.linewidth=10

#
vis.smooth_window = 10
vis.lockout = False
vis.xvalidation = 10
vis.sliding_window = 30

# 
vis.window = 15
vis.cbar_thick = 0.05
vis.alpha = 1.0
    
# 
vis.min_trials = 10
# 
vis.ctr_plot = 0
    

# 
vis.animal_id= 'IA1'
session = 'Feb1_'

print ("   session: ", session)
vis.session_id = session
vis.cbar_offset = 0



   session:  Feb1_


In [10]:
###########################################
########## BEHAVIOR RASTER PLOTS ##########
###########################################

# Fig 1-I - SUPPLEMENTARY FIGURE 1.7

no_movement = 3  # window of behavior lockout

#animal_ids = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']
animal_id = 5

# 
times_array = get_times_array_behavior(animal_id, no_movement)

# 
behaviors = np.arange(6)
for selected_behavior in behaviors:
    
    #
    (all1, all2, rewarded_scatter, 
     rewarded_vertical_histogram1, rewarded_vertical_histogram2, 
     rewarded_fit, rewarded_scatter, p_value1, 
     rewarded_fit2, rewarded_scatter2, p_value2) = get_behavior_scatter_plots(times_array[selected_behavior], 
                                                                              color_id=selected_behavior,
                                                                              plotting=True)
    print ("all1: ", len(all1))
    plt.xticks([])
    plt.savefig('/home/cat/behavior_'+str(selected_behavior)+'.png')
    plt.close()
    

/home/cat/code/widefieldPredict/Fig1_lever_pull_analysis/Visualize.py:2240: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  idx = np.where(codes==code)[0]


labels:  ['left_paw' 'right_paw' 'nose' 'jaw' 'right_ear' 'tongue' 'lever']
animal_nam:  IA1


<ipython-input-7-b48791f168d8>:746: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  times_array = np.array(times_array)
<ipython-input-7-b48791f168d8>:704: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  all1 = np.array(all1)


NameError: name 'rewarded_vertical_histogram2' is not defined

In [19]:
######################################################################
########## STACKED SCATTER PLOT - HORIZONTAL BODY MOVEMENT ###########
######################################################################

# FIG 1 E,F ...

# 
labels = ['left_paw', 'right_paw', 'nose', 'jaw', 'right_ear', 'tongue', 'lever']

animal_ids = np.arange(6)
selected_behavior = 1
fig = plt.figure(figsize=(2,8))
for animal_id in animal_ids:
    ax=plt.subplot(6,1,animal_id+1)
    
    no_movement = 3  # window of behavior lockout

    # 
    times_array = get_times_array_behavior(animal_id, no_movement)

    print ("len timees_array", len(times_array[0]))
    #
    plot_horizontal_scatter_behavior(times_array,
                                     selected_behavior,
                                     animal_id)

    
plt.suptitle(labels[selected_behavior])
# 
if True:
    plt.savefig('/home/cat/horizontal_'+str(selected_behavior)+'.png',dpi=100)
    plt.close()
else:
    plt.show() 
              
        

<ipython-input-17-61070b95b551>:747: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  times_array = np.array(times_array)
<ipython-input-17-61070b95b551>:704: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  all1 = np.array(all1)
/home/cat/code/widefieldPredict/Fig1_lever_pull_analysis/Visualize.py:2240: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  idx = np.where(codes==code)[0]


len timees_array 30
labels:  ['left_paw' 'right_paw' 'nose' 'jaw' 'right_ear' 'tongue' 'lever']
animal_nam:  IA1


<ipython-input-17-61070b95b551>:747: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  times_array = np.array(times_array)
<ipython-input-17-61070b95b551>:704: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  all1 = np.array(all1)
<ipython-input-17-61070b95b551>:747: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  times_array = np.array(time

len timees_array 12
labels:  ['left_paw' 'right_paw' 'nose' 'jaw' 'right_ear' 'tongue' 'lever']
animal_nam:  IA1
len timees_array 11
labels:  ['left_paw' 'right_paw' 'nose' 'jaw' 'right_ear' 'tongue' 'lever']
animal_nam:  IA1
len timees_array 10
labels:  ['left_paw' 'right_paw' 'nose' 'jaw' 'right_ear' 'tongue' 'lever']
animal_nam:  IA1


<ipython-input-17-61070b95b551>:747: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  times_array = np.array(times_array)
<ipython-input-17-61070b95b551>:704: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  all1 = np.array(all1)
/home/cat/code/widefieldPredict/Fig1_lever_pull_analysis/Visualize.py:2240: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  idx = np.where(codes==code)[0]


len timees_array 11
labels:  ['left_paw' 'right_paw' 'nose' 'jaw' 'right_ear' 'tongue' 'lever']
animal_nam:  IA1
len timees_array 70
labels:  ['left_paw' 'right_paw' 'nose' 'jaw' 'right_ear' 'tongue' 'lever']
animal_nam:  IA1


<ipython-input-17-61070b95b551>:747: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  times_array = np.array(times_array)
<ipython-input-17-61070b95b551>:704: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  all1 = np.array(all1)


In [97]:
#######################################################
######### SLIDING WINDOW TRENDS BODY MOVEMENTS ########
#######################################################

# Fig 1 J
# 
labels = ['left_paw', 'right_paw', 'nose', 'jaw', 'right_ear', 'tongue', 'lever']
cmaps = ['Greens','Blues',"Oranges", "Reds", "Greys", "Purples"]
colors = ['green','blue','orange','red','grey','purple']

#
no_movement = 3  # window of behavior lockout
plot_data= False

#    
animal_ids = np.arange(6)
selected_behavior = 0

selected_behaviors = np.arange(6)
for selected_behavior in selected_behaviors:

    # 
    fig = plt.figure(figsize=(2,8))
    animal_ids= [0,5]
    for animal_id in animal_ids:
        ax=plt.subplot(6,1,animal_id+1)


        # 
        times_array = get_times_array_behavior(animal_id, no_movement)


        #
        coefs, n_trials = compute_trends_intra_session_behavior(times_array,
                                                              selected_behavior,
                                                              animal_id,
                                                               plot_data)
        # 
        x = np.arange(coefs.shape[0])
        plt.scatter(x, 
                    coefs[:,0],
                    c=colors[selected_behavior])

        plt.plot([x[0],x[-1]],[0,0],'--', c='black')
        plt.xlim(x[0],x[-1])
        plt.xticks([])
        plt.yticks([])


    if True:
        plt.savefig('/home/cat/behavior_trends_'+str(selected_behavior)+".png")
        plt.close()
    else:
        plt.show()

IA1pm_Feb1_30Hz
Lever to [Ca] shift:  2.566666666666667
DLC to [Ca] shift:  -4.733333333333352
IA1pm_Feb2_30Hz
Lever to [Ca] shift:  2.5
DLC to [Ca] shift:  -9.8
IA1pm_Feb3_30Hz
Lever to [Ca] shift:  2.033333333333333
DLC to [Ca] shift:  4.1999999999999496
IA1pm_Feb4_30Hz
Lever to [Ca] shift:  2.4
DLC to [Ca] shift:  -10.0
IA1pm_Feb5_30Hz
Lever to [Ca] shift:  2.2666666666666666
DLC to [Ca] shift:  -9.533333333333335
IA1pm_Feb9_30Hz
Lever to [Ca] shift:  2.3333333333333335
DLC to [Ca] shift:  2.7999999999999545
IA1pm_Feb10_30Hz
Lever to [Ca] shift:  2.7333333333333334
DLC to [Ca] shift:  -4.466666666666686
IA1pm_Feb11_30Hz
Lever to [Ca] shift:  2.533333333333333
DLC to [Ca] shift:  4.733333333333281
IA1pm_Feb12_30Hz
Lever to [Ca] shift:  2.466666666666667
DLC to [Ca] shift:  -0.46666666666670054
IA1pm_Feb15_30Hz
Lever to [Ca] shift:  2.6666666666666665
DLC to [Ca] shift:  4.266666666666616
IA1pm_Feb16_30Hz
Lever to [Ca] shift:  2.5
DLC to [Ca] shift:  4.599999999999948
IA1pm_Feb17_30Hz

<ipython-input-58-bec35d50b2d6>:747: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  times_array = np.array(times_array)
<ipython-input-58-bec35d50b2d6>:706: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  all1 = np.array(all1)


Lever to [Ca] shift:  3.3
DLC to [Ca] shift:  -10.0
AQ2pm_Dec10_30Hz
Lever to [Ca] shift:  1.8
DLC to [Ca] shift:  -10.0
AQ2am_Dec11_30Hz
Lever to [Ca] shift:  2.1666666666666665
DLC to [Ca] shift:  -6.6666666666666785
AQ2pm_Dec14_30Hz
Lever to [Ca] shift:  2.6
DLC to [Ca] shift:  -5.600000000000016
AQ2am_Dec14_30Hz
Lever to [Ca] shift:  2.3333333333333335
DLC to [Ca] shift:  -10.0
AQ2pm_Dec16_30Hz
Lever to [Ca] shift:  2.2666666666666666
DLC to [Ca] shift:  2.4666666666666224
AQ2am_Dec17_30Hz
Lever to [Ca] shift:  2.066666666666667
DLC to [Ca] shift:  -5.466666666666683
AQ2pm_Dec17_30Hz
Lever to [Ca] shift:  2.2
DLC to [Ca] shift:  -7.266666666666676
AQ2am_Dec18_30Hz
Lever to [Ca] shift:  2.7
DLC to [Ca] shift:  -9.06666666666667
AQ2pm_Dec18_30Hz
Lever to [Ca] shift:  2.533333333333333
DLC to [Ca] shift:  4.799999999999947
AQ2am_Dec21_30Hz
Lever to [Ca] shift:  2.533333333333333
DLC to [Ca] shift:  -8.466666666666672
AQ2am_Dec22_30Hz
Lever to [Ca] shift:  2.5
DLC to [Ca] shift:  -10.0

Lever to [Ca] shift:  2.4
DLC to [Ca] shift:  -10.0
IA1pm_Feb5_30Hz
Lever to [Ca] shift:  2.2666666666666666
DLC to [Ca] shift:  -9.533333333333335
IA1pm_Feb9_30Hz
Lever to [Ca] shift:  2.3333333333333335
DLC to [Ca] shift:  2.7999999999999545
IA1pm_Feb10_30Hz
Lever to [Ca] shift:  2.7333333333333334
DLC to [Ca] shift:  -4.466666666666686
IA1pm_Feb11_30Hz
Lever to [Ca] shift:  2.533333333333333
DLC to [Ca] shift:  4.733333333333281
IA1pm_Feb12_30Hz
Lever to [Ca] shift:  2.466666666666667
DLC to [Ca] shift:  -0.46666666666670054
IA1pm_Feb15_30Hz
Lever to [Ca] shift:  2.6666666666666665
DLC to [Ca] shift:  4.266666666666616
IA1pm_Feb16_30Hz
Lever to [Ca] shift:  2.5
DLC to [Ca] shift:  4.599999999999948
IA1pm_Feb17_30Hz
Lever to [Ca] shift:  2.8333333333333335
DLC to [Ca] shift:  4.799999999999947
IA1pm_Feb18_30Hz
IA1pm_Feb19_30Hz
IA1pm_Feb22_30Hz
IA1pm_Feb23_30Hz
IA1pm_Feb24_30Hz
IA1pm_Feb25_30Hz
IA1pm_Feb26_30Hz
IA1pm_Feb29_30Hz
IA1pm_Mar1_30Hz
IA1pm_Mar2_30Hz
IA1pm_Mar3_30Hz
IA1am_Mar

Lever to [Ca] shift:  2.1
DLC to [Ca] shift:  -10.0
AQ2am_Jan27_30Hz
Lever to [Ca] shift:  3.066666666666667
DLC to [Ca] shift:  -5.33333333333335
AQ2pm_Jan27_30Hz
Lever to [Ca] shift:  2.533333333333333
DLC to [Ca] shift:  4.1999999999999496
AQ2am_Jan28_30Hz
Lever to [Ca] shift:  2.8
DLC to [Ca] shift:  4.733333333333281
AQ2pm_Jan28_30Hz
Lever to [Ca] shift:  2.066666666666667
DLC to [Ca] shift:  3.199999999999953
AQ2am_Jan29_30Hz
Lever to [Ca] shift:  3.1333333333333333
DLC to [Ca] shift:  5.666666666666611
AQ2pm_Jan29_30Hz
Lever to [Ca] shift:  2.6
DLC to [Ca] shift:  4.799999999999947
AQ2am_Feb2_30Hz
Lever to [Ca] shift:  2.4
DLC to [Ca] shift:  4.066666666666617
AQ2am_Feb3_30Hz
Lever to [Ca] shift:  2.8666666666666667
DLC to [Ca] shift:  4.466666666666615
AQ2am_Feb4_30Hz
Lever to [Ca] shift:  3.0
DLC to [Ca] shift:  4.799999999999947
AQ2am_Feb5_30Hz
Lever to [Ca] shift:  2.966666666666667
DLC to [Ca] shift:  5.199999999999946
AQ2am_Feb9_30Hz
Lever to [Ca] shift:  3.133333333333333

...

COEFS:  [array([0.00117118, 0.08369671]), array([ 0.00126062, -0.0287064 ]), array([0.0012841, 0.1033188]), array([0.00077706, 0.41739341]), array([3.11940966e-04, 8.04990310e-01]), array([6.11024150e-04, 9.67175388e-01]), array([0.0021534 , 0.77858527]), array([0.00241503, 0.87209302]), array([0.00273983, 1.07957849]), array([0.00183084, 1.80220446]), array([1.62399374e-03, 1.89014050e+00]), array([2.64423077e-04, 2.76610950e+00]), array([7.47428444e-04, 2.87076066e+00]), array([-2.59950805e-04,  4.96741764e+00]), array([3.52191413e-04, 5.32146318e+00]), array([-4.47786225e-04,  6.12827035e+00]), array([1.49262075e-04, 5.56964632e+00]), array([-1.41435599e-04,  5.61906492e+00]), array([7.32893560e-04, 5.26005329e+00]), array([6.33944544e-04, 5.51041667e+00]), array([5.33318426e-04, 5.71535853e+00]), array([-8.44141324e-04,  6.84326550e+00]), array([-2.05724508e-04,  5.80038760e+00]), array([-1.61001789e-04,  5.97286822e+00]), array([5.06484794e-04, 4.80256783e+00]), array([4.79092129e

Lever to [Ca] shift:  2.533333333333333
DLC to [Ca] shift:  4.799999999999947
AQ2am_Dec21_30Hz
Lever to [Ca] shift:  2.533333333333333
DLC to [Ca] shift:  -8.466666666666672
AQ2am_Dec22_30Hz
Lever to [Ca] shift:  2.5
DLC to [Ca] shift:  -10.0
AQ2am_Dec23_30Hz
Lever to [Ca] shift:  1.9333333333333333
DLC to [Ca] shift:  -10.0
AQ2am_Dec28_30Hz
Lever to [Ca] shift:  2.7
DLC to [Ca] shift:  3.8666666666666174
AQ2am_Dec29_30Hz
Lever to [Ca] shift:  2.8666666666666667
DLC to [Ca] shift:  -8.800000000000004
AQ2am_Dec30_30Hz
Lever to [Ca] shift:  0.0
DLC to [Ca] shift:  1.333333333333293
AQ2am_Dec31_30Hz
AQ2am_Jan4_30Hz
Lever to [Ca] shift:  2.2333333333333334
DLC to [Ca] shift:  3.333333333333286
AQ2am_Jan5_30Hz
Lever to [Ca] shift:  1.7333333333333334
DLC to [Ca] shift:  3.799999999999951
AQ2am_Jan6_30Hz
Lever to [Ca] shift:  1.5333333333333334
DLC to [Ca] shift:  2.733333333333288
AQ2am_Jan7_30Hz
Lever to [Ca] shift:  3.0
DLC to [Ca] shift:  -5.666666666666682
AQ2am_Jan8_30Hz
Lever to [Ca] 

Lever to [Ca] shift:  2.4
DLC to [Ca] shift:  -10.0
IA1pm_Feb5_30Hz
Lever to [Ca] shift:  2.2666666666666666
DLC to [Ca] shift:  -9.533333333333335
IA1pm_Feb9_30Hz
Lever to [Ca] shift:  2.3333333333333335
DLC to [Ca] shift:  2.7999999999999545
IA1pm_Feb10_30Hz
Lever to [Ca] shift:  2.7333333333333334
DLC to [Ca] shift:  -4.466666666666686
IA1pm_Feb11_30Hz
Lever to [Ca] shift:  2.533333333333333
DLC to [Ca] shift:  4.733333333333281
IA1pm_Feb12_30Hz
Lever to [Ca] shift:  2.466666666666667
DLC to [Ca] shift:  -0.46666666666670054
IA1pm_Feb15_30Hz
Lever to [Ca] shift:  2.6666666666666665
DLC to [Ca] shift:  4.266666666666616
IA1pm_Feb16_30Hz
Lever to [Ca] shift:  2.5
DLC to [Ca] shift:  4.599999999999948
IA1pm_Feb17_30Hz
Lever to [Ca] shift:  2.8333333333333335
DLC to [Ca] shift:  4.799999999999947
IA1pm_Feb18_30Hz
IA1pm_Feb19_30Hz
IA1pm_Feb22_30Hz
IA1pm_Feb23_30Hz
IA1pm_Feb24_30Hz
IA1pm_Feb25_30Hz
IA1pm_Feb26_30Hz
IA1pm_Feb29_30Hz
IA1pm_Mar1_30Hz
IA1pm_Mar2_30Hz
IA1pm_Mar3_30Hz
IA1am_Mar

Lever to [Ca] shift:  2.1
DLC to [Ca] shift:  -10.0
AQ2am_Jan27_30Hz
Lever to [Ca] shift:  3.066666666666667
DLC to [Ca] shift:  -5.33333333333335
AQ2pm_Jan27_30Hz
Lever to [Ca] shift:  2.533333333333333
DLC to [Ca] shift:  4.1999999999999496
AQ2am_Jan28_30Hz
Lever to [Ca] shift:  2.8
DLC to [Ca] shift:  4.733333333333281
AQ2pm_Jan28_30Hz
Lever to [Ca] shift:  2.066666666666667
DLC to [Ca] shift:  3.199999999999953
AQ2am_Jan29_30Hz
Lever to [Ca] shift:  3.1333333333333333
DLC to [Ca] shift:  5.666666666666611
AQ2pm_Jan29_30Hz
Lever to [Ca] shift:  2.6
DLC to [Ca] shift:  4.799999999999947
AQ2am_Feb2_30Hz
Lever to [Ca] shift:  2.4
DLC to [Ca] shift:  4.066666666666617
AQ2am_Feb3_30Hz
Lever to [Ca] shift:  2.8666666666666667
DLC to [Ca] shift:  4.466666666666615
AQ2am_Feb4_30Hz
Lever to [Ca] shift:  3.0
DLC to [Ca] shift:  4.799999999999947
AQ2am_Feb5_30Hz
Lever to [Ca] shift:  2.966666666666667
DLC to [Ca] shift:  5.199999999999946
AQ2am_Feb9_30Hz
Lever to [Ca] shift:  3.133333333333333

Lever to [Ca] shift:  2.2
DLC to [Ca] shift:  -10.0
IA1am_May4_Week4_30Hz
IA1am_May5_Week4_30Hz
Lever to [Ca] shift:  3.1333333333333333
DLC to [Ca] shift:  -8.466666666666672
IA1am_May6_Week4_30Hz
Lever to [Ca] shift:  2.066666666666667
DLC to [Ca] shift:  4.066666666666617
IA1am_May9_Week5_30Hz
Lever to [Ca] shift:  2.2
DLC to [Ca] shift:  4.93333333333328
IA1am_May10_Week5_30Hz
Lever to [Ca] shift:  2.3
DLC to [Ca] shift:  3.9999999999999503
IA1am_May12_Week5_30Hz
Lever to [Ca] shift:  2.066666666666667
DLC to [Ca] shift:  -2.6000000000000263
IA1am_May13_Week5_30Hz
Lever to [Ca] shift:  2.4
DLC to [Ca] shift:  -5.400000000000016
IA1am_May16_Week6_30Hz
Lever to [Ca] shift:  2.533333333333333
DLC to [Ca] shift:  2.3333333333332895
IA1am_May17_Week6_30Hz
Lever to [Ca] shift:  2.0
DLC to [Ca] shift:  3.666666666666618
IA1am_May18_Week6_30Hz
Lever to [Ca] shift:  2.3666666666666667
DLC to [Ca] shift:  -1.2666666666666977
IA1am_May20_Week6_30Hz
Lever to [Ca] shift:  2.2
DLC to [Ca] shift:

In [16]:
#####################################################################
####### SCATTER PLOTS # OF BODY MOVEMENT PER SESSION PER ANIMAL #####
#####################################################################

# Sup Fig 1.8

cmaps = ['Greys_r','Purples_r','Blues_r',"Greens_r","Reds_r",
         "Oranges_r","PuRd_r","PuBu_r",
         "YlGn_r","RdPu_r"]


labels = ['left_paw', 
        'right_paw',
        'nose',
        'jaw',
        'right_ear',
        'tongue',
        'lever']

clrs = ['green','blue','yellow','cyan','red','magenta','black']

# 
session_id = 'all'
animal_id = 'IA1'
animal_ids = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']
animal_ids = ["IA1"]
no_movement = 3

#
rsq_array = []

first_last = []
mse_array = []

######################################
######################################
######################################
fig=plt.figure(figsize=(10,6))
for a, animal_id in enumerate(animal_ids):
    ax=plt.subplot(2,3,a+1)
    sessions = get_sessions(vis.main_dir,
                            animal_id,
                            session_id)
    
    # 
    rsq_array.append([])
    first_last.append([])
    mse_array.append([])
    
    # 
    ctr_sess=0
    locs = []
    for k in range(7):
        locs.append([])
    for session in sessions:
        print (session)
        temp_, code_04_times, feature_quiescent = load_trial_times_whole_stack(
                                                                         vis.main_dir,
                                                                         animal_id,
                                                                         session,
                                                                         no_movement)
        
        if feature_quiescent is not None:
            for k in range(len(feature_quiescent)):
                if a==1 and k==5:
                    continue
            
                temp = np.array(feature_quiescent[k])
                plt.scatter(ctr_sess, temp.shape[0],
                            s=100,
                            c=clrs[k],
                            edgecolor='black',
                            alpha=.3)
                
                locs[k].append([ctr_sess,temp.shape[0]])
                
            ctr_sess+=1
    # 
    for k in range(6):
        temp = np.array(locs[k])
        if a==1 and k==5:
            first_last[a].append(np.nan)
            continue
        
        try:
            coef, rsq, corr_pearson, diff, mse = polyfit(temp[:,0],temp[:,1],1)
            print (a,k," rsq: ", rsq, "  Pearson corr: ", corr_pearson)

            #coef = np.polyfit(temp[:,0],temp[:,1],1)
            poly1d_fn = np.poly1d(coef) 

            rsq_array[a].append(rsq)
            first_last[a].append(diff)
            mse_array[a].append(mse)

            # poly1d_fn is now a function which takes in x and returns an estimate for y
            x=np.arange(len(locs[k]))
            Y = poly1d_fn(x)
            plt.plot(x, Y, linewidth=5, c=clrs[k], label=str(round(corr_pearson,2))) #labels[k])
        except:
            pass
    
    plt.ylim(0,225)
    #plt.xlim(0,x[-1])
    plt.xticks([])
    plt.yticks([])
    #if a==0:
    #plt.legend(fontsize=16)        
                
    print ('')
plt.show()

IA1pm_Feb1_30Hz
Lever to [Ca] shift:  2.566666666666667
DLC to [Ca] shift:  -4.733333333333352
IA1pm_Feb2_30Hz
Lever to [Ca] shift:  2.5
DLC to [Ca] shift:  -9.8
IA1pm_Feb3_30Hz
Lever to [Ca] shift:  2.033333333333333
DLC to [Ca] shift:  4.1999999999999496
IA1pm_Feb4_30Hz
Lever to [Ca] shift:  2.4
DLC to [Ca] shift:  -10.0
IA1pm_Feb5_30Hz
Lever to [Ca] shift:  2.2666666666666666
DLC to [Ca] shift:  -9.533333333333335
IA1pm_Feb9_30Hz
Lever to [Ca] shift:  2.3333333333333335
DLC to [Ca] shift:  2.7999999999999545
IA1pm_Feb10_30Hz
Lever to [Ca] shift:  2.7333333333333334
DLC to [Ca] shift:  -4.466666666666686
IA1pm_Feb11_30Hz
Lever to [Ca] shift:  2.533333333333333
DLC to [Ca] shift:  4.733333333333281
IA1pm_Feb12_30Hz
Lever to [Ca] shift:  2.466666666666667
DLC to [Ca] shift:  -0.46666666666670054
IA1pm_Feb15_30Hz
Lever to [Ca] shift:  2.6666666666666665
DLC to [Ca] shift:  4.266666666666616
IA1pm_Feb16_30Hz
Lever to [Ca] shift:  2.5
DLC to [Ca] shift:  4.599999999999948
IA1pm_Feb17_30Hz

In [119]:
##############################
####### RSQUARE ANALYSIS #####
##############################

# Sup Fig 1.8

fig=plt.figure()
ax=plt.subplot(121)
for p in range(len(rsq_array)):
    arr = []
    for k in range(len(rsq_array[p])):
        print (k,p,rsq_array[p][k])
        temp = rsq_array[p][k]
        arr.append(temp)
        plt.scatter(p, temp, 
                    s=300,
                    edgecolor='black',
                    c=clrs[p],
                    alpha=.2)
    arr=np.array(arr)
    print (arr)
    plt.scatter(p, np.nanmean(arr,0), 
                s=800,
                edgecolor='black',
                c=clrs[p],
               alpha=1)    
    
# ax=plt.subplot(132)
# for p in range(len(rmse_array)):
#     arr = []
#     for k in range(len(rmse_array[p])):
#         temp = np.sqrt(mse_array[p][k])
#         arr.append(temp)
#         plt.scatter(p, temp, 
#                     s=100,
#                     edgecolor='black',
#                     c=clrs[p],
#                     alpha=.2)
#     arr=np.array(arr)
#     print (arr)
#     plt.scatter(p, np.nanmean(arr,0), 
#                 s=300,
#                 edgecolor='black',
#                 c=clrs[p],
#                alpha=1)    
plt.xlim(-0.1,5.1)
plt.ylim(0,1.0)
plt.xticks([])
plt.yticks([])

ax=plt.subplot(122)
for p in range(len(rsq_array)):
    arr = []
    for k in range(len(rsq_array[p])):
        temp = first_last[p][k]
        arr.append(temp)
        plt.scatter(p, temp, 
                    s=300,
                    edgecolor='black',
                    c=clrs[p],
                    alpha=.2)
    arr=np.array(arr)
    print (arr)
    plt.scatter(p, np.nanmean(arr,0), 
                s=800,
                edgecolor='black',
                c=clrs[p],
               alpha=1)       
plt.plot([-0.1,5.1],[0,0],'--',c='black',linewidth=5,alpha=.5)
plt.xlim(-0.1,5.1)
plt.ylim(-100,100)
plt.xticks([])
plt.yticks([])
plt.show()

0 0 0.11619049376105949
1 0 0.006296256295899559
2 0 0.19169811822809382
3 0 0.3220963839014383
4 0 0.020690802238381885
5 0 0.004620009798426293
[0.11619049 0.00629626 0.19169812 0.32209638 0.0206908  0.00462001]
0 1 0.0044965637443398765
1 1 0.014501805570249427
2 1 0.024550676719134912
3 1 0.0014118733336292194
4 1 0.030899959370329227
[0.00449656 0.01450181 0.02455068 0.00141187 0.03089996]
0 2 0.0016143457651371658
1 2 0.0006045105789351439
2 2 0.11874000681502415
3 2 0.16347554054428176
4 2 0.20323133699473983
5 2 0.16282859571889136
[0.00161435 0.00060451 0.11874001 0.16347554 0.20323134 0.1628286 ]
0 3 0.005511088556071228
1 3 0.15817722624330452
2 3 0.0001940094235809273
3 3 0.003505639070848288
4 3 0.3801782413569438
5 3 0.024141204420908315
[5.51108856e-03 1.58177226e-01 1.94009424e-04 3.50563907e-03
 3.80178241e-01 2.41412044e-02]
0 4 0.009521611552537026
1 4 0.0036939890710383275
2 4 0.1631670315921181
3 4 0.8260402391927234
4 4 0.8489306269043384
5 4 0.22056652529606702
[

In [104]:
print (rmse_array)

[[565.5555951056729, 673.7410752688172, 672.7945618588553, 504.3188629341243, 690.0919886293412, 1310.8811617846989], [987.7813714063714, 638.7397047397048, 1187.8466394716395, 476.4652292152293, 1160.6318958818958], [184.51157024793395, 923.6231404958679, 555.718181818182, 998.8793388429754, 453.2099173553719, 1643.9140495867769], [1382.429090909091, 271.100606060606, 976.0206060606058, 422.0751515151516, 1202.3054545454547, 2989.2896969696967], [496.5652892561984, 150.68099173553722, 442.20743801652895, 184.6446280991736, 169.63966942148753, 1111.794214876033], [1099.8564959946013, 746.5112118674781, 1623.933135708662, 1125.3112328630168, 925.4060467150732, 995.292356874164]]


In [109]:
# TEST

times = np.load('/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb2_30Hz/IA1pm_Feb2_30Hz_abstimes.npy')
pos = np.load('/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb1_30Hz/IA1pm_Feb1_30Hz_abspositions.npy')

chunks = get_movements_lever_pos(pos,
                                 times)
print (np.array(chunks)[:10])

print (chunks[:,1]-chunks[:,0])


idx:  (139,)
lever   # of quiescent periods:  138
[[0.26710892 0.27502704]
 [0.27502704 0.28320885]
 [0.28320885 0.29155397]
 [0.29155397 0.30801296]
 [0.30801296 0.31631207]
 [0.31631207 0.324301  ]
 [0.324301   0.33246207]
 [0.33246207 0.340698  ]
 [0.340698   7.28061558]
 [7.28061558 7.28894892]]
[7.91811943e-03 8.18181038e-03 8.34512711e-03 1.64589882e-02
 8.29911232e-03 7.98892975e-03 8.16106796e-03 8.23593140e-03
 6.93991758e+00 8.33333333e-03 8.33333333e-03 8.33333333e-03
 8.33333333e-03 8.33333333e-03 3.89740100e+01 8.33333333e-03
 8.33333333e-03 8.33333333e-03 8.33333333e-03 8.33333333e-03
 8.33333333e-03 8.33333333e-03 8.33333333e-03 8.33333333e-03
 8.33333333e-03 8.33333333e-03 1.82649083e+02 8.33333333e-03
 8.33333333e-03 8.33333333e-03 8.33333333e-03 8.33333333e-03
 8.33333333e-03 8.33333333e-03 5.00000000e-02 8.33333333e-03
 8.33333333e-03 8.33333333e-03 2.50000000e-01 8.33333333e-03
 1.56416667e+01 8.33333333e-03 4.16666667e-02 8.33333333e-03
 8.33333333e-03 8.33333333e-

In [50]:
#############################################
######### MAKE A MOVIE WITH DLC TRACES ######
#############################################

# #
# fname_traces = ''
# traces = np.load(fname_traces)
print (traces.shape)

# 
fname_video = '/media/cat/4TBSSD/yuki/IA1/vids/prestroke/IA1pm_Feb1_30Hz.mp4'
movements = np.zeros((traces.shape[0],traces.shape[1]),'int32')

# 
start = 0*15+1
end = start+60*15
make_video_dlc(traces_original,
               movements,
               fname_video,
               start,
               end)

  3%|▎         | 30/900 [00:00<00:02, 293.48it/s]

(7, 20059, 3)
Traces:  (7, 20059, 3)
loadin gmovie:  /media/cat/4TBSSD/yuki/IA1/vids/prestroke/IA1pm_Feb1_30Hz.mp4
Frame size read:  (360, 640, 3)


100%|██████████| 900/900 [00:03<00:00, 286.22it/s]


In [30]:
d= np.load('/media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2pm_Jan18_30Hz/AQ2pm_Jan18_30Hz_whole_stack_trial_ROItimeCourses_15sec_pca30components.npy')
print (d.shape)

(40000, 30)
